In [1]:
import pandas as pd
import mysql.connector
from mysql.connector import Error

In [2]:
# GET MySQL password from personal_vault.txt
file_path = 'personal_vault.txt'

with open(file_path, 'r') as file:
    password = file.read().strip()


def create_mysql_connection(host_name, user_name, user_password):
    connection = None
    try:
        connection = mysql.connector.connect(
            host = host_name,
            user = user_name,
            password = user_password
        )
        print("Connected successfully")

    except Error as e:
        print(e, 'Failed to connect')
    
    return connection

mydb = create_mysql_connection('localhost', 'root', password)


Connected successfully


In [3]:
def create_database(mydb, query):
    cursor = mydb.cursor()
    try:
        cursor.execute(query)
        print("DB Successfully created")

    except Error as e:
        print(e, 'Failed to connect')
    
query = "CREATE DATABASE IF NOT EXISTS bookstore_pos"
create_database(mydb, query)

DB Successfully created


In [4]:
def connect_to_db(host_name, user_name, user_password, user_db):
    connection = None
    try:
        connection = mysql.connector.connect(
            host = host_name,
            user = user_name,
            password = user_password,
            database = user_db
        )
        print("Connected successfully")

    except Error as e:
        print(e, 'Failed to connect')
    
    return connection
connect_to_db('localhost', 'root', password, 'bookstore_pos')

Connected successfully


In [5]:
def execute_sql(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query successfully executed")

    except Error as e:
        print(e, 'Failed to connect')

Create Data from Appendix A into MySQL

In [6]:
#Create Customer Table
create_customer_table = """
CREATE TABLE if not exists customers (
	id int primary key,
    name varchar(80) not null,
    email varchar(80),
    tel varchar(15),
    create_at timestamp default current_timestamp not null,
    updated_at timestamp default current_timestamp on update current_timestamp not null
);
"""
connection = connect_to_db('localhost', 'root', password, 'bookstore_pos')
execute_sql(connection, create_customer_table)

Connected successfully
Query successfully executed


In [7]:
#Insert Customer Table
insert_customer_data = """
INSERT INTO customers (
	id,
    name,
    email,
    tel,
    create_at,
    updated_at
) VALUES 
(1, "Irfan Bakti", "irfan88@example.com", "+60123456789", "2019-08-07 08:13:21", "2019-08-07 08:13:21"),
(2, 'Jack Smmith', 'jack.smmith@acme.io', '+60132456781', '2019-08-07 08:13:21', '2019-08-07 08:13:21'),
(3, 'Nazir','','+601185434012', '2019-08-07 08:13:21', '2019-08-07 08:13:21'),
(4, 'Faiz Ma', 'faiz.ma@jholow.cn', '+6019772002', '2019-08-07 08:13:21', '2019-08-07 08:13:21'),
(5, 'Isham Rais', 'isham@pmo.gov.my', '+60135482020', '2019-08-07 08:13:21', '2019-08-07 08:13:21'),
(6, 'Shanon Teoh', 'shahnon.teoh@st.com.sg','','2019-08-07 08:13:21', '2019-08-07 08:13:21')
;
"""
execute_sql(connection, insert_customer_data)

Query successfully executed


In [8]:
# Create and Insert Invoice
create_table_invoice = """
CREATE TABLE if not exists invoices (
	id int primary key not null,
    number int(8) not null,
    sub_total decimal(8,2) not null,
    tax_total decimal(5,2) not null,
    total decimal(8,2) not null,
    customer_id int references customers(id),
    create_at timestamp default current_timestamp,
    updated_at timestamp default current_timestamp on update current_timestamp
);
"""

insert_invoice_data = """
INSERT INTO invoices (
	id,
    number,
    sub_total,
    tax_total,
    total,
    customer_id,
    create_at,
    updated_at
) VALUE 
(1, 20190001, 30.00, 0.00, 30.00, 1, "2019-08-07 08:13:21", "2019-08-07 08:13:21"),
(2, 20190002, 150.00, 0.00, 150.00, 2, '2019-08-07 08:13:21', '2019-08-07 08:13:21'),
(3, 20190003, 30.55, 0.00, 30.00, 2, '2019-09-15 08:13:21', '2019-08-07 08:13:21'),
(4, 20190004, 55.45, 0.00, 55.00, 3, '2019-09-15 08:13:21', '2019-08-07 08:13:21'),
(5, 20190005, 450.00, 0.00, 00.00, 6, '2019-09-15 08:13:21', '2019-08-07 08:13:21')
;
"""

connection = connect_to_db('localhost', 'root', password, 'bookstore_pos')
execute_sql(connection, create_table_invoice)
execute_sql(connection, insert_invoice_data)

Connected successfully
Query successfully executed
Query successfully executed


In [9]:
# Create and Insert Invoice Lines
create_table_invoice_lines = """
CREATE TABLE if not exists invoice_lines (
	id int primary key not null,
    description varchar(200) not null,
    unit_price decimal(5,2)not null,
    quantity int not null,
    sub_total decimal(8,2) not null,
    tax_total decimal(5,2) not null,
    total decimal(8,2) not null,
    tax_id int null,
    sku_id int not null,
    invoice_id int references invoices(id)
);
"""

insert_invoice_line_data = """
INSERT INTO invoice_lines (
	id,
    description,
    unit_price,
    quantity,
    sub_total,
    tax_total,
    total,
    tax_id,
    sku_id,
    invoice_id
) VALUE 
(1, 'Book #1', 30.00, 1, 30.00, 0.00, 30.00, NULL, 1, 1),
(2, 'Book #2', 25.00, 4, 100.00, 0.00, 100.00, NULL, 2, 2),
(3, 'Book #3', 50.00, 1, 50.00, 0.00, 50.00, NULL, 3, 2),
(4, 'Book #4', 30.00, 1, 30.00, 0.00, 30.00, NULL, 1, 3),
(5, 'Book #5', 30.00, 1, 30.00, 0.00, 30.00, NULL, 1, 4),
(6, 'Book #6', 25.00, 1, 25.00, 0.00, 25.00, NULL, 2, 4),
(7, 'Book #7', 30.00, 5, 150.00, 0.00, 150.00, NULL, 1, 5),
(8, 'Book #8', 50.00, 6, 300.00, 0.00, 300.00, NULL, 3, 5)
;
"""

connection = connect_to_db('localhost', 'root', password, 'bookstore_pos')
execute_sql(connection, create_table_invoice_lines)
execute_sql(connection, insert_invoice_line_data)

Connected successfully
Query successfully executed
Query successfully executed


In [10]:
def sql_pool_query(connection, query):
    cursor = connection.cursor()
    result = None

    try: 
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    
    except Error as e:
        print(e, 'Failed to connect')

Query all necessary data then save to excel

In [17]:
# Test Read All SQL
read_query = """
SELECT *
FROM 
customers
LEFT JOIN invoices ON customers.id = invoices.customer_id
LEFT JOIN invoice_lines ON invoices.id = invoice_lines.invoice_id
"""
connection = connect_to_db('localhost', 'root', password, 'bookstore_pos')
get_results = sql_pool_query(connection, read_query)
get_results

pd.read_sql_query(read_query, con = connection)

Connected successfully


C:\Users\hpute\AppData\Local\Temp\ipykernel_18532\3363132938.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(read_query, con = connection)


,id,name,email,tel,create_at,updated_at,id,number,sub_total,tax_total,...,id,description,unit_price,quantity,sub_total,tax_total,total,tax_id,sku_id,invoice_id
0,1,Irfan Bakti,irfan88@example.com,+60123456789,2019-08-07 08:13:21,2019-08-07 08:13:21,1.0,20190001.0,30.00,0.0,...,1.0,Book #1,30.0,1.0,30.0,0.0,30.0,None,1.0,1.0
1,2,Jack Smmith,jack.smmith@acme.io,+60132456781,2019-08-07 08:13:21,2019-08-07 08:13:21,3.0,20190003.0,30.55,0.0,...,4.0,Book #4,30.0,1.0,30.0,0.0,30.0,None,1.0,3.0
2,2,Jack Smmith,jack.smmith@acme.io,+60132456781,2019-08-07 08:13:21,2019-08-07 08:13:21,2.0,20190002.0,150.00,0.0,...,3.0,Book #3,50.0,1.0,50.0,0.0,50.0,None,3.0,2.0
3,2,Jack Smmith,jack.smmith@acme.io,+60132456781,2019-08-07 08:13:21,2019-08-07 08:13:21,2.0,20190002.0,150.00,0.0,...,2.0,Book #2,25.0,4.0,100.0,0.0,100.0,None,2.0,2.0
4,3,Nazir,,+601185434012,2019-08-07 08:13:21,2019-08-07 08:13:21,4.0,20190004.0,55.45,0.0,...,6.0,Book #6,25.0,1.0,25.0,0.0,25.0,None,2.0,4.0
5,3,Nazir,,+601185434012,2019-08-07 08:13:21,2019-08-07 08:13:21,4.0,20190004.0,55.45,0.0,...,5.0,Book #5,30.0,1.0,30.0,0.0,30.0,None,1.0,4.0
6,4,Faiz Ma,faiz.ma@jholow.cn,+6019772002,2019-08-07 08:13:21,2019-08-07 08:13:21,NaN,NaN,NaN,NaN,...,NaN,None,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
7,5,Isham Rais,isham@pmo.gov.my,+60135482020,2019-08-07 08:13:21,2019-08-07 08:13:21,NaN,NaN,NaN,NaN,...,NaN,None,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
8,6,Shanon Teoh,shahnon.teoh@st.com.sg,,2019-08-07 08:13:21,2019-08-07 08:13:21,5.0,20190005.0,450.00,0.0,...,8.0,Book #8,50.0,6.0,300.0,0.0,300.0,None,3.0,5.0
9,6,Shanon Teoh,shahnon.teoh@st.com.sg,,2019-08-07 08:13:21,2019-08-07 08:13:21,5.0,20190005.0,450.00,0.0,...,7.0,Book #7,30.0,5.0,150.0,0.0,150.0,None,1.0,5.0


In [11]:
question2_b = """
SELECT customers.name, count(*) as number_of_customers
FROM 
customers
LEFT JOIN invoices ON customers.id = invoices.customer_id
LEFT JOIN invoice_lines ON invoices.id = invoice_lines.invoice_id
WHERE invoice_lines.id IS NOT NULL
GROUP BY customers.name
HAVING number_of_customers > 5;

"""

connection = connect_to_db('localhost', 'root', password, 'bookstore_pos')
get_results = sql_pool_query(connection, question2_b)
get_results 

columns = ['customer_name']
data_b = pd.DataFrame(get_results, columns=columns)
data_b.to_excel('output_data.xlsx', sheet_name= 'question2_b', index= False)
data_b

Connected successfully


,customer_name


In [12]:
question2_c = """
SELECT invoice_lines.sku_id, invoice_lines.description, customers.name
FROM 
customers
LEFT JOIN invoices ON customers.id = invoices.customer_id
LEFT JOIN invoice_lines ON invoices.id = invoice_lines.invoice_id
WHERE invoices.id IS NOT NULL;

"""

connection = connect_to_db('localhost', 'root', password, 'bookstore_pos')
get_results = sql_pool_query(connection, question2_c)
get_results

columns = ['sku_id', 'description', 'customer_name']
data_c = pd.DataFrame(get_results, columns=columns)
data_c.to_excel('output_data.xlsx', sheet_name= 'question2_c', index= False)
data_c

Connected successfully


,sku_id,description,customer_name
0,1,Book #1,Irfan Bakti
1,3,Book #3,Jack Smmith
2,2,Book #2,Jack Smmith
3,1,Book #4,Jack Smmith
4,2,Book #6,Nazir
5,1,Book #5,Nazir
6,3,Book #8,Shanon Teoh
7,1,Book #7,Shanon Teoh


In [13]:
question2_d = """
SELECT customers.name
FROM 
customers
LEFT JOIN invoices ON customers.id = invoices.customer_id
LEFT JOIN invoice_lines ON invoices.id = invoice_lines.invoice_id
WHERE invoice_lines.id IS NULL
GROUP BY customers.name;

"""

connection = connect_to_db('localhost', 'root', password, 'bookstore_pos')
get_results = sql_pool_query(connection, question2_d)
get_results 

columns = ['customer_name']
data_d = pd.DataFrame(get_results, columns=columns)
data_d.to_excel('output_data.xlsx', sheet_name= 'question2_d', index= False)
data_d

Connected successfully


,customer_name
0,Faiz Ma
1,Isham Rais
